In [7]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import numpy as np
import importlib
import sklearn
import random
import torch
import math

import socialSig
importlib.reload(socialSig)
from helpers import *

In [8]:
####### Load our Data
#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'
devSet = pd.read_csv("./us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)
devSet = devSet.drop(['sending'], axis = 1)

print(devSet.head())

y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values

mMScale = preprocessing.MinMaxScaler()
X = mMScale.fit_transform(X)

   sending_citizen_unspecified  sending_citizenship_unknown  \
0                  1134.995083                   243.827586   
1                   917.067746                   600.000000   
2                   637.977315                   557.150000   
3                  1178.378744                  1678.562500   
4                  2231.128863                  2916.538705   

   sending_household_not_owned  sending_household_owned  \
0                  1178.209016              1106.832815   
1                  1274.160656               795.118020   
2                   729.454795               610.791414   
3                  1560.244328              1084.109969   
4                  2104.943216              2278.834178   

   sending_household_owned_unknown  sending_indigeneity  sending_internet  \
0                       736.562500           918.960526       2111.233685   
1                        -1.000000            -1.000000       1988.103175   
2                       468.392857 

In [9]:
len(X[0])

29

In [10]:
def train_test_split(X, y, split):

    train_num = int(len(X) * split)
    val_num = int(len(X) - train_num)

    train_indices = random.sample(range(len(X)), train_num)
    val_indices = [i for i in range(len(X)) if i not in train_indices]

    x_train, x_val = X[train_indices], X[val_indices]
    y_train, y_val = y[val_indices], y[val_indices]

    return x_train, y_train, x_val, y_val

In [11]:
####### Build and fit the Model
lr = 1e-6
batchSize = 50
model = socialSig.SocialSigNet(X=X, outDim = batchSize)
epochs = 1

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [12]:
x_train, y_train, x_val, y_val = train_test_split(X, y, .80)

train = [(k,v) for k,v in zip(x_train, y_train)]
val = [(k,v) for k,v in zip(x_val, y_val)]

train = torch.utils.data.DataLoader(train, batch_size = batchSize, shuffle = True)
val = torch.utils.data.DataLoader(val, batch_size = batchSize, shuffle = True)

In [30]:
best_mae = 9000000000000000000
best_model_wts = deepcopy(model.state_dict())


for epoch in range(epochs):


    for phase in ['train','val']:



        if phase == 'train':

            c = 1
            running_train_mae, running_train_loss = 0, 0

            # print("In training")

            for inputs, output in train:

                if len(inputs) == batchSize:

                    # print(c)
                    c += 1

                    inputs = torch.tensor(inputs, dtype = torch.float32, requires_grad = True)
                    output = torch.reshape(torch.tensor(output, dtype = torch.float32, requires_grad = True), (batchSize,1))

                    # Forward pass
                    y_pred = model(inputs, str(epoch) + str(c))
                    loss = criterion(y_pred, output)  
                    
                    # Zero gradients, perform a backward pass, and update the weights.
                    optimizer.zero_grad()
                    grad = torch.autograd.grad(outputs = loss, inputs = inputs, retain_graph = True)
                    loss.backward()
                    optimizer.step()

                    # Update the coordinate weights
                    # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
                    with torch.no_grad():
                        for name, p in model.named_parameters():
                            if name == 'SocialSig.W':
                                new_val = update_function(p, grad[0], loss, lr)
                                p.copy_(new_val)

                    running_train_mae += mae(y_pred, output).item()
                    running_train_loss += loss.item()

        if phase == 'val':

            c = 1
            running_val_mae, running_val_loss,  = 0, 0

            # print("In validation")

            for inputs, output in val:

                if len(inputs) == batchSize:

                    # print(c)
                    c += 1

                    inputs = torch.tensor(inputs, dtype = torch.float32, requires_grad = True)
                    output = torch.reshape(torch.tensor(output, dtype = torch.float32, requires_grad = True), (batchSize,1))

                    # Forward pass
                    y_pred = model(inputs, 1)
                    loss = criterion(y_pred, output)  

                    running_val_mae += mae(y_pred, output).item()
                    running_val_loss += loss.item()
                    
                    if mae(y_pred, output).item() < best_mae:
                        best_mae = mae(y_pred, output).item()
                        best_model_wts = deepcopy(model.state_dict())

                    



                    
    print("Epoch: ", epoch)  
    print("  Train:")
    print("    Loss: ", running_train_loss / c)      
    print("    MAE: ", running_train_mae / c)
    print("  Val:")
    print("    Loss: ", running_val_loss / c)      
    print("    MAE: ", running_val_mae / c)

/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print("    W at beginning: ", torch.tensor(self.W))
/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:56: UserWarning: To copy construct from a tensor, it is recommended 

In [31]:
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [33]:
torch.save({
            'epoch': 50,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            }, "./trained_weights_nosending5.torch")